# Lesson 4: Vertical and horizontal scaling

This lesson is about making Python faster, in two dimensions:

<center>
<img src="../img/horizontal-and-vertical-scaling.svg" width="60%">
</center>

## Vertical scaling

Reminder: Python is slow.

<img src="../img/benchmark-games-2023.svg" width="100%">

We have already seen that NumPy (and Awkward Array) can circumvent Python's slowness by doing computationally intensive work in compiled code.

In [ ]:
import numpy as np
import awkward as ak

events = ak.from_parquet("../data/SMHiggsToZZTo4L.parquet")[:100000]

<br>

In [ ]:
%%timeit -r1 -n1

pz = [[muon.pt * np.sinh(muon.eta) for muon in event.muon] for event in events]

<br>

In [ ]:
%%timeit -r1 -n1

pz = events.muon.pt * np.sinh(events.muon.eta)

Array-oriented programming connects Python with compiled code, but it's not the only way to do that.

<img src="../img/history-of-bindings-2.svg" width="100%">

Although much faster than pure Python, array-oriented techniques are not as fast as imperative, compiled code.

In [ ]:
%%writefile quadratic_formula_c.c

#include <math.h>

void run(double* a, double* b, double* c, double* output) {
    for (int i = 0;  i < 1000000;  i++) {
        output[i] = (-b[i] + sqrt(b[i]*b[i] - 4*a[i]*c[i])) / (2*a[i]);
    }
}

In [ ]:
!cc quadratic_formula_c.c -shared -lm -o quadratic_formula_c.so

<br>

In [ ]:
import ctypes

quadratic_formula_c = ctypes.CDLL("./quadratic_formula_c.so")
quadratic_formula_c.run.argtypes = (ctypes.POINTER(ctypes.c_double),) * 4
quadratic_formula_c.run.restype = None

In [ ]:
a = np.random.uniform(5, 10, 1000000)
b = np.random.uniform(10, 20, 1000000)
c = np.random.uniform(-0.1, 0.1, 1000000)

<br>

In [ ]:
output = np.zeros(1000000, dtype=np.float64)
quadratic_formula_c.run(*[arg.ctypes.data_as(ctypes.POINTER(ctypes.c_double)) for arg in (a, b, c, output)])
output

<br>

In [ ]:
%%timeit

quadratic_formula_c.run(*[arg.ctypes.data_as(ctypes.POINTER(ctypes.c_double)) for arg in (a, b, c, output)])

<br>

In [ ]:
%%timeit

(-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

Why? NumPy allocates a new array for each intermediate step.

* Memory allocation is expensive (`malloc` has to search for unused memory).
* Accessing different memory is expensive (the CPU can't re-use its cache, and acessing RAM is much slower than most mathematical operations).

To calculate the first expression, NumPy takes the steps shown in the second:

<br>

In [ ]:
(-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

<br>

In [ ]:
tmp1 = np.negative(b)            # -b
tmp2 = np.square(b)              # b**2
tmp3 = np.multiply(4, a)         # 4*a
tmp4 = np.multiply(tmp3, c)      # tmp3*c
del tmp3
tmp5 = np.subtract(tmp2, tmp4)   # tmp2 - tmp4
del tmp2, tmp4
tmp6 = np.sqrt(tmp5)             # sqrt(tmp5)
del tmp5
tmp7 = np.add(tmp1, tmp6)        # tmp1 + tmp6
del tmp1, tmp6
tmp8 = np.multiply(2, a)         # 2*a
np.divide(tmp7, tmp8)            # tmp7 / tmp8     This is the result!

Best way to make C++ extensions for Python: pybind11

In [ ]:
%%writefile quadratic_formula_pybind11.cpp

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
namespace py = pybind11;

void run(py::array_t<double, py::array::c_style | py::array::forcecast> a_numpy,
         py::array_t<double, py::array::c_style | py::array::forcecast> b_numpy,
         py::array_t<double, py::array::c_style | py::array::forcecast> c_numpy,
         py::array_t<double> output_numpy) {
    const double* a = a_numpy.data();
    const double* b = b_numpy.data();
    const double* c = c_numpy.data();
    double* output = output_numpy.mutable_data();
    for (int i = 0;  i < output_numpy.size();  i++) {
        output[i] = (-b[i] + sqrt(b[i]*b[i] - 4*a[i]*c[i])) / (2*a[i]);
    }
}

PYBIND11_MODULE(quadratic_formula_pybind11, m) {
    m.def("run", &run);
}

In [ ]:
import os
import sys
from pybind11 import get_include

inc = "-I " + get_include()
plat = "-undefined dynamic_lookup" if "darwin" in sys.platform else "-fPIC"
pyinc = !python3-config --cflags

<br>

In [ ]:
!c++ -std=c++11 quadratic_formula_pybind11.cpp -shared {inc} {pyinc.s} -o quadratic_formula_pybind11.so {plat}

In [ ]:
import quadratic_formula_pybind11

In [ ]:
output = np.zeros(1000000, dtype=np.float64)
quadratic_formula_pybind11.run(a, b, c, output)
output

In [ ]:
%%timeit

quadratic_formula_pybind11.run(a, b, c, output)

<br>

In [ ]:
%%timeit

(-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

Important! Put the loop over big data on the _compiled_ side of the boundary.

In [ ]:
%%writefile quadratic_formula_pybind11_noloop.cpp

#include <pybind11/pybind11.h>
namespace py = pybind11;

double run(double a, double b, double c) {
    return (-b + sqrt(b*b - 4*a*c)) / (2*a);
}
PYBIND11_MODULE(quadratic_formula_pybind11_noloop, m) {
    m.def("run", &run);
}

In [ ]:
!c++ -std=c++11 quadratic_formula_pybind11_noloop.cpp -shared {inc} {pyinc.s} -o quadratic_formula_pybind11_noloop.so {plat}

<br>

In [ ]:
import quadratic_formula_pybind11_noloop

<br>

In [ ]:
%%timeit

output = [quadratic_formula_pybind11_noloop.run(a_i, b_i, c_i) for a_i, b_i, c_i in zip(a, b, c)]

Leaving Python, writing C++ code, and then importing it* is fine for a long-term project, like a library that will be used many times, but it's inconvenient in the middle of a data analysis.

<br>

*If we change the C++, recompile, and do

```python
import quadratic_formula_pybind11
```

again, we will _not_ get the new version. We would still have the old version, with no error messages or warnings!

Numba is a Just In Time (JIT) compiler for _imperative_ Python.

In [ ]:
import numba as nb

<br>

In [ ]:
@nb.njit
def quadratic_formula_numba(a, b, c):
    output = np.empty(len(a), dtype=np.float64)
    for i, (a_i, b_i, c_i) in enumerate(zip(a, b, c)):
        output[i] = (-b_i + np.sqrt(b_i**2 - 4*a_i*c_i)) / (2*a_i)
    return output

quadratic_formula_numba(a, b, c)

<br>

In [ ]:
%%timeit

quadratic_formula_numba(a, b, c)

JAX is a JIT compiler for _array-oriented_ Python.

In [ ]:
import jax

<br>

In [ ]:
@jax.jit
def quadratic_formula_jax(a, b, c):
    return (-b + jax.numpy.sqrt(b**2 - 4*a*c)) / (2*a)

quadratic_formula_jax(a, b, c)

<br>

In [ ]:
%%timeit

quadratic_formula_jax(a, b, c)

Note: it's hard to get JAX to use only one thread or use the CPU when a GPU is available, so use caution when interpreting performance results.

<center>
<img src="../img/slow-fast-imperative-vectorized.svg" width="80%">
</center>

Imperative versus vectorized (array-oriented) coding style is independent of slow versus fast.

<br><br>

Why would you want to pick one over the other?

* array-oriented style is simpler for certain types of problems, fits interactive data analysis cadence

* array-oriented style is more complicated or even impossible for some problems

<img src="../img/numerical-recipes.jpg" width="200px" align="right">

Suppose you need to calculate some special functions (and can't use SciPy).

<br>

Consider the log of the Gamma function,

$$\log\Gamma(z) \hspace{0.5 cm} \mbox{where} \hspace{0.5 cm} \Gamma(z) = \int_0^\infty t^{z - 1} e^{-t} \, dt$$

<br clear="right">

Page 214, converted from Pascal to Python:

In [ ]:
def log_of_gamma(x):
    tmp = x + 5.5
    tmp -= (x + 0.5) * np.log(tmp)
    series = 1.000000000190015
    for i, coefficient in enumerate([
        76.18009172947146, -86.50532032941677, 24.01409824083091,
        -1.231739572450155, 0.1208650973866179e-2, -0.5395239384953e-5,
    ]):
        series += coefficient / (x + i + 1)
    return -tmp + np.log(2.5066282746310005 * series / x)

Every line is a [closed-form](https://en.wikipedia.org/wiki/Closed-form_expression) expression (the loop over six coefficients could be "unrolled" into six lines), so this is very easy to vectorize.

In [ ]:
log_of_gamma(0.1), log_of_gamma(1), log_of_gamma(5), log_of_gamma(10)

<br>

In [ ]:
log_of_gamma(np.array([0.1, 1, 5, 10]))

<br>

In [ ]:
import matplotlib.pyplot as plt
import scipy.special

x = 10**np.linspace(-3, 1, 100)
plt.plot(x, log_of_gamma(x));
plt.scatter(x, scipy.special.loggamma(x), color="darkorange");

Now suppose that you want to compute the incomplete gamma $P(a, x)$.

$$P(a, x) = \frac{1}{\Gamma(a)} \int_0^x t^{a - 1} e^{-t} \, dt \hspace{0.5 cm} \mbox{where} \hspace{0.5 cm} a > 0$$

Page 219, converted from Pascal to Python:

In [ ]:
SMALL_NUMBER = 3e-7

def incomplete_gamma_P(a, x):
    gln = log_of_gamma(a)

    delta = summation = 1.0 / a

    for i in range(100):
        delta *= x / (a + i + 1)
        summation += delta
        if np.absolute(delta) < np.absolute(summation) * SMALL_NUMBER:
            return summation * np.exp(-x + a*np.log(x) - gln)

    raise RuntimeError("did not converge")

Although this can be computed for scalar inputs,

In [ ]:
plt.plot(x, [incomplete_gamma_P(0.5, x_i) for x_i in x]);
plt.plot(x, [incomplete_gamma_P(2, x_i) for x_i in x]);
plt.plot(x, [incomplete_gamma_P(10, x_i) for x_i in x]);
plt.scatter(x, scipy.special.gammainc(0.5, x));
plt.scatter(x, scipy.special.gammainc(2, x));
plt.scatter(x, scipy.special.gammainc(10, x));

It doesn't "just work" for arrays:

In [ ]:
incomplete_gamma_P(3.0, x)

<br><br>

Python can't decide whether to enter the body of the `if` statement or not because the expression may be true for some elements of the array and false for others.

**What should we do?**

Have all values continue to iterate, even if some have already converged?

In [ ]:
def incomplete_gamma_P_numpy_keep_going(a, x):
    gln = log_of_gamma(a)

    delta = summation = 1.0 / a

    for i in range(100):
        delta *= x / (a + i + 1)
        summation += delta

    return summation * np.exp(-x + a*np.log(x) - gln)

In [ ]:
plt.plot(x, incomplete_gamma_P_numpy_keep_going(0.5, x));
plt.plot(x, incomplete_gamma_P_numpy_keep_going(2.0, x));
plt.plot(x, incomplete_gamma_P_numpy_keep_going(10, x));

**What should we do?**

Keep track of which values have converged and don't compute those?

In [ ]:
def incomplete_gamma_P_numpy_tricky_bookkeeping(a, x):
    gln = log_of_gamma(a)

    delta = np.full(x.shape, 1.0 / a)
    summation = delta.copy()

    not_converged = np.ones(x.shape, np.bool_)

    for i in range(100):
        delta[not_converged] *= x[not_converged] / (a + i + 1)
        summation[not_converged] += delta[not_converged]
        not_converged &= np.absolute(delta) >= np.absolute(summation) * SMALL_NUMBER

    return summation * np.exp(-x + a*np.log(x) - gln)

In [ ]:
plt.plot(x, incomplete_gamma_P_numpy_tricky_bookkeeping(0.5, x));
plt.plot(x, incomplete_gamma_P_numpy_tricky_bookkeeping(2.0, x));
plt.plot(x, incomplete_gamma_P_numpy_tricky_bookkeeping(10, x));

**What should we do?**

Just write some imperative code?

In [ ]:
log_of_gamma_numba = nb.njit(log_of_gamma)

@nb.vectorize
def incomplete_gamma_P_numba(a, x):
    gln = log_of_gamma_numba(a)

    delta = summation = 1.0 / a

    for i in range(100):
        delta *= x / (a + i + 1)
        summation += delta
        if np.absolute(delta) < np.absolute(summation) * SMALL_NUMBER:
            return summation * np.exp(-x + a*np.log(x) - gln)

    raise RuntimeError("did not converge")

incomplete_gamma_P_numba(3.0, np.array([0.001, 1, 2, 10]))

In [ ]:
plt.plot(x, incomplete_gamma_P_numba(0.5, x));
plt.plot(x, incomplete_gamma_P_numba(2, x));
plt.plot(x, incomplete_gamma_P_numba(10, x));

So... which one's faster?

In [ ]:
%%timeit

incomplete_gamma_P_numpy_keep_going(3.0, x)

<br>

In [ ]:
%%timeit

incomplete_gamma_P_numpy_tricky_bookkeeping(3.0, x)

<br>

In [ ]:
%%timeit

incomplete_gamma_P_numba(3.0, x)

<br>

Tricky bookkeeping usually isn't helpful—it's usually better to let array-oriented code do unnecessary calculations than try to keep track of which calcuations are necessary. But imperative code lets you do exactly what you want.

The exercises are on vertical scaling, so do one now, before I talk about horizontal scaling.

## Horizontal scaling